In [ ]:
import pandas as pd
import numpy as np
from collections import Counter
import nltk
from nltk.corpus import stopwords
import string 
import re 
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
from nltk.tokenize import WordPunctTokenize

In [ ]:
df = pd.read_csv('reddit_posts.csv')

In [ ]:
df.info()

In [ ]:
print (df.text.isnull().sum())
df = df.loc[~df.text.isnull()]

In [ ]:
df.head()

### Defining lables

In [ ]:
split_it = df['labels'].str.split(';')
labels = []

for i in split_it:
    z = Counter(i).most_common()
    labels.append(z[0][0])

df['label_updated'] = labels

In [ ]:
df['label_updated'].value_counts()

In [ ]:
my_labels = ['announcement', 'elaboration', 'humor', 'appreciation', 'question',
       'answer', 'agreement', 'negativereaction', 'other', 'disagreement']
plt.figure(figsize=(10,4))
df.label_updated.value_counts().plot(kind='bar');

### Cleaning text data

In [ ]:
def print_example(index):
    example = df[df.index == index][['text', 'label_updated']].values[0]
    if len(example) > 0:
        print(example[0])
        print('label_updated:', example[1])
print_example(9)

In [ ]:
#TODO: move to txt 
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", 
                   "can't've": "cannot have", "'cause": "because", "could've": "could have", 
                   "couldn't": "could not", "couldn't've": "could not have","didn't": "did not", 
                   "doesn't": "does not", "don't": "do not", "hadn't": "had not", 
                   "hadn't've": "had not have", "hasn't": "has not", "haven't": "have not", 
                   "he'd": "he would", "he'd've": "he would have", "he'll": "he will", 
                   "he'll've": "he will have", "he's": "he is", "how'd": "how did", 
                   "how'd'y": "how do you", "how'll": "how will", "how's": "how is", 
                   "I'd": "I would", "I'd've": "I would have", "I'll": "I will", 
                   "I'll've": "I will have","I'm": "I am", "I've": "I have", 
                   "i'd": "i would", "i'd've": "i would have", "i'll": "i will", 
                   "i'll've": "i will have","i'm": "i am", "i've": "i have", 
                   "isn't": "is not", "it'd": "it would", "it'd've": "it would have", 
                   "it'll": "it will", "it'll've": "it will have","it's": "it is", 
                   "let's": "let us", "ma'am": "madam", "mayn't": "may not", 
                   "might've": "might have","mightn't": "might not","mightn't've": "might not have", 
                   "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", 
                   "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", 
                   "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not",
                   "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", 
                   "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", 
                   "she's": "she is", "should've": "should have", "shouldn't": "should not", 
                   "shouldn't've": "should not have", "so've": "so have","so's": "so as", 
                   "this's": "this is",
                   "that'd": "that would", "that'd've": "that would have","that's": "that is", 
                   "there'd": "there would", "there'd've": "there would have","there's": "there is", 
                       "here's": "here is",
                   "they'd": "they would", "they'd've": "they would have", "they'll": "they will", 
                   "they'll've": "they will have", "they're": "they are", "they've": "they have", 
                   "to've": "to have", "wasn't": "was not", "we'd": "we would", 
                   "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", 
                   "we're": "we are", "we've": "we have", "weren't": "were not", 
                   "what'll": "what will", "what'll've": "what will have", "what're": "what are", 
                   "what's": "what is", "what've": "what have", "when's": "when is", 
                   "when've": "when have", "where'd": "where did", "where's": "where is", 
                   "where've": "where have", "who'll": "who will", "who'll've": "who will have", 
                   "who's": "who is", "who've": "who have", "why's": "why is", 
                   "why've": "why have", "will've": "will have", "won't": "will not", 
                   "won't've": "will not have", "would've": "would have", "wouldn't": "would not", 
                   "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would",
                   "y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                   "you'd": "you would", "you'd've": "you would have", "you'll": "you will", 
                   "you'll've": "you will have", "you're": "you are", "you've": "you have" } 

In [ ]:
tokenizer = WordPunctTokenizer()
pattern1 = r'@[A-Za-z0-9]+'
pattern2 = r'https?://[A-Za-z0-9./]+'
pattern_combined = r'|'.join((pattern1, pattern2))

def reddit_cleaner_v1(text):
    #link removal
    soup = BeautifulSoup(text, 'html.parser')
    souped = soup.get_text()
    stripped = re.sub(pattern_combined, '', souped)
    apostrophe_handled = re.sub("’", "'", stripped)
    expanded = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in apostrophe_handled.split(" ")])    letters_only = re.sub("[^a-zA-Z]", " ", expanded)
    lower_case = letters_only.lower()
    words = tokenizer.tokenize(lower_case)
    return (" ".join(words)).strip()

# testing = df.text[:100]

# test_result = []
# for i in testing:
#     test_result.append(reddit_cleaner_v1(i))
# test_result

In [ ]:
cleaned = []
for i in df.text.iteritems():
    cleaned.append(reddit_cleaner_v1(i[1])) 
df['cleaned_text'] =  cleaned 

In [ ]:
df.to_csv('processed_v1.csv')